<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Tensorflow/Tensorflow_FineTuning_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [4]:
inputs = tf.keras.Input(shape=(None, None, 3))
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (160, 160)))(inputs)
x = tf.keras.layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input)(x)

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    weights = 'imagenet', input_tensor=x,input_shape=(160,160,3),
    include_top=False, pooling='avg'
)

model = tf.keras.Sequential([
                            base_model,
                            tf.keras.layers.Dense(10, activation = 'softmax')
])

model.summary()

9412608/9406464 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Functi (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 2,270,794
Trainable params: 2,236,682
Non-trainable params: 34,112
_________________________________________________________________


In [5]:

base_model.trainable = False

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Functi (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 2,270,794
Trainable params: 12,810
Non-trainable params: 2,257,984
_________________________________________________________________


In [6]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
            loss = 'sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [7]:
print(model.evaluate(x_test, y_test,verbose=0))

[2.6643471717834473, 0.1363999992609024]


In [8]:
model.fit(x_train, y_train, epochs = 6, validation_split = 0.2, batch_size = 256)

Epoch 1/6
157/157 [==============================] - 58s 372ms/step - loss: 1.6337 - accuracy: 0.4559 - val_loss: 1.0657 - val_accuracy: 0.6659
Epoch 2/6
157/157 [==============================] - 58s 370ms/step - loss: 0.8579 - accuracy: 0.7336 - val_loss: 0.7125 - val_accuracy: 0.7720
Epoch 3/6
157/157 [==============================] - 58s 370ms/step - loss: 0.6484 - accuracy: 0.7918 - val_loss: 0.5972 - val_accuracy: 0.7998
Epoch 4/6
157/157 [==============================] - 58s 369ms/step - loss: 0.5657 - accuracy: 0.8144 - val_loss: 0.5426 - val_accuracy: 0.8174
Epoch 5/6
157/157 [==============================] - 58s 369ms/step - loss: 0.5206 - accuracy: 0.8274 - val_loss: 0.5088 - val_accuracy: 0.8257
Epoch 6/6
157/157 [==============================] - 58s 370ms/step - loss: 0.4913 - accuracy: 0.8355 - val_loss: 0.4881 - val_accuracy: 0.8329


In [9]:
print(model.evaluate(x_test, y_test))

313/313 [==============================] - 13s 43ms/step - loss: 0.4987 - accuracy: 0.8314
[0.4987438917160034, 0.8313999772071838]


In [10]:
layer_names = [l.name for l in base_model.layers]

idx = layer_names.index('block_12_expand')
print(idx)

110


In [11]:
import pprint
pprint.pprint(layer_names)

['input_1',
 'lambda',
 'lambda_1',
 'Conv1_pad',
 'Conv1',
 'bn_Conv1',
 'Conv1_relu',
 'expanded_conv_depthwise',
 'expanded_conv_depthwise_BN',
 'expanded_conv_depthwise_relu',
 'expanded_conv_project',
 'expanded_conv_project_BN',
 'block_1_expand',
 'block_1_expand_BN',
 'block_1_expand_relu',
 'block_1_pad',
 'block_1_depthwise',
 'block_1_depthwise_BN',
 'block_1_depthwise_relu',
 'block_1_project',
 'block_1_project_BN',
 'block_2_expand',
 'block_2_expand_BN',
 'block_2_expand_relu',
 'block_2_depthwise',
 'block_2_depthwise_BN',
 'block_2_depthwise_relu',
 'block_2_project',
 'block_2_project_BN',
 'block_2_add',
 'block_3_expand',
 'block_3_expand_BN',
 'block_3_expand_relu',
 'block_3_pad',
 'block_3_depthwise',
 'block_3_depthwise_BN',
 'block_3_depthwise_relu',
 'block_3_project',
 'block_3_project_BN',
 'block_4_expand',
 'block_4_expand_BN',
 'block_4_expand_relu',
 'block_4_depthwise',
 'block_4_depthwise_BN',
 'block_4_depthwise_relu',
 'block_4_project',
 'block_4_pr

In [12]:
base_model.trainable = True
for layer in base_model.layers[:idx]:
    layer.trainable = False

In [13]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.00001),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Functi (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 2,270,794
Trainable params: 1,812,426
Non-trainable params: 458,368
_________________________________________________________________


In [15]:
model.fit(x_train, y_train, epochs = 6, validation_split = 0.2, batch_size = 256)

Epoch 1/6
157/157 [==============================] - 77s 492ms/step - loss: 0.5841 - accuracy: 0.8077 - val_loss: 0.4705 - val_accuracy: 0.8351
Epoch 2/6
157/157 [==============================] - 76s 486ms/step - loss: 0.3880 - accuracy: 0.8695 - val_loss: 0.4465 - val_accuracy: 0.8481
Epoch 3/6
157/157 [==============================] - 76s 486ms/step - loss: 0.3199 - accuracy: 0.8923 - val_loss: 0.4122 - val_accuracy: 0.8608
Epoch 4/6
157/157 [==============================] - 76s 486ms/step - loss: 0.2739 - accuracy: 0.9087 - val_loss: 0.3893 - val_accuracy: 0.8667
Epoch 5/6
157/157 [==============================] - 76s 486ms/step - loss: 0.2386 - accuracy: 0.9208 - val_loss: 0.3701 - val_accuracy: 0.8732
Epoch 6/6
157/157 [==============================] - 76s 486ms/step - loss: 0.2100 - accuracy: 0.9319 - val_loss: 0.3521 - val_accuracy: 0.8782


In [16]:
path_to_zip = tf.keras.utils.get_file(
    fname='cats_and_dogs_filtered.zip',
    origin='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip',
    extract=True
)


68608000/68606236 [==============================] - 2s 0us/step


In [18]:
import sys, os
path_to_dir = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(path_to_dir, 'train')
test_dir = os.path.join(path_to_dir, 'validation')

In [20]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split = 0.2
)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [21]:
batch_size = 64
height = 160
width = 160

train_generator = train_datagen.flow_from_directory(
    batch_size = batch_size,
    directory=train_dir,
    target_size = (height, width),
    class_mode = 'binary',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    batch_size = batch_size,
    directory = train_dir,
    target_size = (height, width),
    class_mode = 'binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    batch_size = batch_size,
    directory = test_dir,
    target_size=(height, width),
    class_mode = 'binary'
)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [22]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    weights='imagenet',
    input_shape=(height,width, 3),
    include_top = False,
    pooling='avg'
)

x = base_model.output
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=base_model.input, outputs = x)

In [23]:
print(len(model.layers))

157


In [24]:
print(model.layers[0].trainable)

True


In [26]:
base_model.trainable = False

In [27]:
print(model.layers[0].trainable)

False


In [28]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [29]:
print(model.evaluate(test_generator, verbose = 0))

[0.7505014538764954, 0.5270000100135803]


In [30]:
model.fit(
    train_generator,
    steps_per_epoch = train_generator.n // batch_size,
    validation_data = valid_generator,
    validation_steps = valid_generator.n // batch_size,
    epochs = 6
)

Epoch 1/6
25/25 [==============================] - 7s 288ms/step - loss: 0.6344 - accuracy: 0.6425 - val_loss: 0.5717 - val_accuracy: 0.7057
Epoch 2/6
25/25 [==============================] - 7s 279ms/step - loss: 0.5106 - accuracy: 0.7675 - val_loss: 0.4716 - val_accuracy: 0.8203
Epoch 3/6
25/25 [==============================] - 7s 281ms/step - loss: 0.4238 - accuracy: 0.8450 - val_loss: 0.3894 - val_accuracy: 0.8880
Epoch 4/6
25/25 [==============================] - 7s 279ms/step - loss: 0.3552 - accuracy: 0.8831 - val_loss: 0.3302 - val_accuracy: 0.9193
Epoch 5/6
25/25 [==============================] - 7s 279ms/step - loss: 0.3006 - accuracy: 0.9162 - val_loss: 0.2778 - val_accuracy: 0.9375
Epoch 6/6
25/25 [==============================] - 7s 279ms/step - loss: 0.2575 - accuracy: 0.9256 - val_loss: 0.2427 - val_accuracy: 0.9453


In [31]:
print(model.evaluate(test_generator))

16/16 [==============================] - 3s 211ms/step - loss: 0.2465 - accuracy: 0.9320
[0.24652345478534698, 0.9319999814033508]


In [32]:
idx = [l.name for l in base_model.layers].index('block_12_expand')

for layer in base_model.layers[idx:]:
    layer.trainable = True
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.00001),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [33]:
model.fit(
    train_generator,
    steps_per_epoch = train_generator.n // batch_size,
    validation_data = valid_generator,
    validation_steps = valid_generator.n // batch_size,
    epochs = 6
)

Epoch 1/6
25/25 [==============================] - 8s 321ms/step - loss: 0.3932 - accuracy: 0.8300 - val_loss: 0.1538 - val_accuracy: 0.9609
Epoch 2/6
25/25 [==============================] - 7s 281ms/step - loss: 0.2380 - accuracy: 0.9169 - val_loss: 0.1225 - val_accuracy: 0.9661
Epoch 3/6
25/25 [==============================] - 7s 278ms/step - loss: 0.1575 - accuracy: 0.9650 - val_loss: 0.1057 - val_accuracy: 0.9609
Epoch 4/6
25/25 [==============================] - 7s 279ms/step - loss: 0.1146 - accuracy: 0.9737 - val_loss: 0.0933 - val_accuracy: 0.9609
Epoch 5/6
25/25 [==============================] - 7s 284ms/step - loss: 0.0867 - accuracy: 0.9869 - val_loss: 0.0872 - val_accuracy: 0.9609
Epoch 6/6
25/25 [==============================] - 7s 280ms/step - loss: 0.0661 - accuracy: 0.9906 - val_loss: 0.0713 - val_accuracy: 0.9688


In [35]:
print(model.evaluate(test_generator))

16/16 [==============================] - 3s 210ms/step - loss: 0.0721 - accuracy: 0.9740
[0.07206118851900101, 0.9739999771118164]
